# 알라딘

**알라딘 OpenAPI 이용 안내**
* https://blog.aladin.co.kr/openapi/popup/5353304

**알라딘 OpenAPI 매뉴얼**
* https://docs.google.com/document/d/1mX-WxuoGs8Hy-QalhHcvuV17n50uGI2Sg_GHofgiePE/edit?tab=t.0


## 상품 조회 API

**상품 리스트 API**
* title : API 결과의 제목
* link : API 결과와 관련된 알라딘 페이지 URL 주소
* author : 저자/아티스트
* pubDate : 출간일(출시일)
* description : 상품설명(요약)
* isbn : 10자리 ISBN
* isbn13 : 13자리 ISBN
* itemId : 상품을 구분짓는 유일한 값
* priceSales : 판매가
* priceStandard : 정가
* mallType : 상품의 몰타입(국내도서:BOOK, 음반:MUSIC, Dvd:DVD, 외서:FOREIGN, 전자책:EBOOK, 중고상품:USED)
* stockStatus : 재고상태(정상유통일 경우 비어있음, 품절/절판 등)
* mileage : 마일리지
* cover : 커버(표지)
* categoryId : 카테고리 ID
* categoryName : 카테고리 명
* publisher : 출판사(제작사/출시사)
* salesPoint : 판매지수
* adult : 성인 등급 여부 (true인 경우 성인 등급 도서)
* fixedPrice : (종이책/전자책인 경우) 정가제 여부 (true인 경우 정가제 해당 도서)
* customerReviewRank : 회원 리뷰 평점(별점 평균) : 0~10점(별0.5개당 1점)
* bestRank : (베스트셀러인 경우만 노출) 베스트셀러 순위 정보
* seriesInfo : 시리즈 정보
* subInfo : 부가정보(이하 결과값은 모두 subInfo의 하위 정보)
* bestDuration : (베스트셀러인 경우만 노출) 베스트셀러 순위 관련 추가 정보

**Options**
* QueryType
    * ItemNewAll : 신간 전체 리스트
    * ItemNewSpecial : 주목할 만한 신간 리스트
    * ItemEditorChoice : 편집자 추천 리스트(카테고리로만 조회 가능 - 국내도서/음반/외서만 지원)
    * Bestseller : 베스트셀러
    * BlogBest : 블로거 베스트셀러 (국내도서만 조회 가능)
* Start : 검색결과 시작페이지
* MaxResults : 검색결과 한 페이지당 최대 출력 개수
* Output
    * XML : REST XML형식
    * JS : JSON방식

In [1]:
from dotenv import load_dotenv
import os
import requests
import json
import pandas as pd

# .env 파일 로드 (환경 변수에 API 키 설정)
load_dotenv()

# 환경 변수에서 ALADIN API 키 가져오기
ALADIN_API_KEY = os.environ.get("ALADIN_API_KEY")


def check_api_key(api_key):
    """
    API 키 유효성 확인.
    """
    if not api_key:
        raise ValueError("ALADIN API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")


def fetch_aladin_data(api_key, query_type="Bestseller", start=1, max_results=50, output="js"):
    """
    ALADIN API로 데이터를 요청하고 JSON 데이터를 반환.
    """
    # ALADIN API 요청 URL
    url = 'http://www.aladin.co.kr/ttb/api/ItemList.aspx'
    
    # API 요청에 필요한 인증 키를 포함한 파라미터
    params = {
        "ttbkey": api_key,  # 부여받은 TTBKey값
        "QueryType": query_type,  # 리스트 종류
        "SearchTarget": "Book",  # 조회 대상 Mall
        "Start": start,  # 검색결과 시작페이지
        "MaxResults": max_results,  # 검색결과 한 페이지당 최대 출력 개수
        "Output": output,  # 출력방법(js: JSON방식)
        "Version": "20131101",  # 검색API의 Version(날짜형식)을 설정(최신버젼: 20131101)
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # HTTP 상태 코드 확인
        return response.json()
    except requests.RequestException as e:
        print(f"API 요청 실패: {e}")
        return None
    except json.JSONDecodeError:
        print("JSON 파싱 오류: 응답이 올바른 JSON 형식이 아닙니다.")
        return None


def json_to_dataframe(data):
    """
    JSON 데이터를 Pandas DataFrame으로 변환.
    """
    if data and 'item' in data:
        return pd.DataFrame(data['item'])
    print("데이터에 'item' 키가 없거나 데이터가 비어 있습니다.")
    return pd.DataFrame()


def filter_columns(df, columns):
    """
    DataFrame에서 필요한 컬럼만 필터링.
    """
    if not df.empty:
        return df[columns]
    print("DataFrame이 비어 있어 필터링할 수 없습니다.")
    return pd.DataFrame()


if __name__ == "__main__":
    try:
        check_api_key(ALADIN_API_KEY)
        raw_data = fetch_aladin_data(ALADIN_API_KEY)
        df = json_to_dataframe(raw_data)
        
        # 필요한 컬럼 필터링
        columns = ['title', 'itemId', 'isbn', 'isbn13', 'link', 'author', 'description', 'pubDate', 'cover', 
                   'categoryName', 'publisher', 'customerReviewRank', 'bestRank']
        df_book_lists = filter_columns(df, columns)

    except ValueError as e:
        print(e)

In [6]:
df_book_lists.head()

,title,itemId,isbn,isbn13,link,author,description,pubDate,cover,categoryName,publisher,customerReviewRank,bestRank
0,팬텀 버스터즈 2 - S코믹스,355241937,K902036963,9791138432283,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,"네오쇼코 (지은이), 김지혜 (옮긴이)","멤버를 모아, 팬텀 버스터즈로서 제령 활동을 시작한 모가리와 친구들. 인지도를 높...",2025-01-15,https://image.aladin.co.kr/product/35524/19/co...,국내도서>만화>본격장르만화>판타지>액션 판타지,㈜소미미디어,10,1
1,소년이 온다 - 2024 노벨문학상 수상작가,40869703,K662930932,9788936434120,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,한강 (지은이),섬세한 감수성과 치밀한 문장으로 인간 존재의 본질을 탐구해온 작가 한강의 여섯번째 ...,2014-05-19,https://image.aladin.co.kr/product/4086/97/cov...,국내도서>소설/시/희곡>한국소설>2000년대 이후 한국소설,창비,10,2
2,"작별하지 않는다 - 2024 노벨문학상 수상작가, 한강 장편소설",278770576,8954682154,9788954682152,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,한강 (지은이),2016년 &lt;채식주의자&gt;로 인터내셔널 부커상을 수상하고 2018년 &lt...,2021-09-09,https://image.aladin.co.kr/product/27877/5/cov...,국내도서>소설/시/희곡>한국소설>2000년대 이후 한국소설,문학동네,9,3
3,"2025 큰별쌤 최태성의 별★별한국사 한국사능력검정시험 심화(1, 2, 3급) 상 ...",353981757,K432035019,9791138928113,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,최태성 (지은이),오랜 연구와 검증으로 한국사에 기초가 전혀 없는 사람도 단기간에 한능검에 합격할 수...,2024-12-24,https://image.aladin.co.kr/product/35398/17/co...,"국내도서>수험서/자격증>한국사능력검정시험>심화(1,2,3급)",이투스북,8,4
4,채식주의자 (리마스터판) - 2024 노벨문학상 수상작가,291370219,8936434594,9788936434595,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,한강 (지은이),2016년 인터내셔널 부커상을 수상하며 한국문학의 입지를 한단계 확장시킨 한강의 장...,2022-03-28,https://image.aladin.co.kr/product/29137/2/cov...,국내도서>소설/시/희곡>한국소설>2000년대 이후 한국소설,창비,8,5


In [78]:
df_book_lists.iloc[1].description

"섬세한 감수성과 치밀한 문장으로 인간 존재의 본질을 탐구해온 작가 한강의 여섯번째 장편소설. '상처의 구조에 대한 투시와 천착의 서사'를 통해 한강만이 풀어낼 수 있는 방식으로 1980년 5월을 새롭게 조명한다."

In [26]:
df_aladin = df_book_lists.iloc[:10]
df_aladin

,title,itemId,isbn,isbn13,link,author,description,pubDate,cover,categoryName,publisher,customerReviewRank,bestRank
0,팬텀 버스터즈 2 - S코믹스,355241937,K902036963,9791138432283,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,"네오쇼코 (지은이), 김지혜 (옮긴이)","멤버를 모아, 팬텀 버스터즈로서 제령 활동을 시작한 모가리와 친구들. 인지도를 높...",2025-01-15,https://image.aladin.co.kr/product/35524/19/co...,국내도서>만화>본격장르만화>판타지>액션 판타지,㈜소미미디어,10,1
1,소년이 온다 - 2024 노벨문학상 수상작가,40869703,K662930932,9788936434120,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,한강 (지은이),섬세한 감수성과 치밀한 문장으로 인간 존재의 본질을 탐구해온 작가 한강의 여섯번째 ...,2014-05-19,https://image.aladin.co.kr/product/4086/97/cov...,국내도서>소설/시/희곡>한국소설>2000년대 이후 한국소설,창비,10,2
2,"작별하지 않는다 - 2024 노벨문학상 수상작가, 한강 장편소설",278770576,8954682154,9788954682152,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,한강 (지은이),2016년 &lt;채식주의자&gt;로 인터내셔널 부커상을 수상하고 2018년 &lt...,2021-09-09,https://image.aladin.co.kr/product/27877/5/cov...,국내도서>소설/시/희곡>한국소설>2000년대 이후 한국소설,문학동네,9,3
3,"2025 큰별쌤 최태성의 별★별한국사 한국사능력검정시험 심화(1, 2, 3급) 상 ...",353981757,K432035019,9791138928113,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,최태성 (지은이),오랜 연구와 검증으로 한국사에 기초가 전혀 없는 사람도 단기간에 한능검에 합격할 수...,2024-12-24,https://image.aladin.co.kr/product/35398/17/co...,"국내도서>수험서/자격증>한국사능력검정시험>심화(1,2,3급)",이투스북,8,4
4,채식주의자 (리마스터판) - 2024 노벨문학상 수상작가,291370219,8936434594,9788936434595,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,한강 (지은이),2016년 인터내셔널 부커상을 수상하며 한국문학의 입지를 한단계 확장시킨 한강의 장...,2022-03-28,https://image.aladin.co.kr/product/29137/2/cov...,국내도서>소설/시/희곡>한국소설>2000년대 이후 한국소설,창비,8,5
5,"2025 큰별쌤 최태성의 별★별한국사 한국사능력검정시험 심화(1, 2, 3급) 하 ...",353981994,K842035019,9791138928120,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,최태성 (지은이),오랜 연구와 검증으로 한국사에 기초가 전혀 없는 사람도 단기간에 한능검에 합격할 수...,2024-12-24,https://image.aladin.co.kr/product/35398/19/co...,"국내도서>수험서/자격증>한국사능력검정시험>심화(1,2,3급)",이투스북,0,6
6,원피스 110 - 시대의 일렁임,355678823,K802036377,9791142305320,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,오다 에이치로 (지은이),"거병 해적단의 도움으로 미래섬 탈출에 빛줄기를 본 루피 일행이었으나, 오로성이 벽이...",2025-01-23,https://image.aladin.co.kr/product/35567/88/co...,국내도서>만화>본격장르만화>판타지>액션 판타지,대원씨아이(만화),10,7
7,이처럼 사소한 것들,329386807,K472936042,9791130646381,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,"클레어 키건 (지은이), 홍한별 (옮긴이)",2023년 4월 국내에 처음 소개된 『맡겨진 소녀』로 국내 문인들과 문학 독자들의 ...,2023-11-27,https://image.aladin.co.kr/product/32938/68/co...,국내도서>소설/시/희곡>세계의 소설>아일랜드소설,다산책방,9,8
8,흑무경담 2 - S코믹스,355242244,K932036963,9791138485593,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,"토가와 요난 (지은이), 김지혜 (옮긴이)",경성에서 연속 엽기 살인사건을 해결한 검은 옷의 무녀 최월자와 일본인 작가 이와야 ...,2025-01-15,https://image.aladin.co.kr/product/35524/22/co...,국내도서>만화>본격장르만화>호러/스릴러,㈜소미미디어,10,9
9,ETS 토익 정기시험 기출문제집 1000 Vol. 4 Reading (리딩) - 토...,330109498,8917239501,9788917239508,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,ETS (엮은이),정기시험 최신 기출문제 10세트가 수록되어 있다. 시험에 나온 최신 기출문제로 실전...,2023-12-18,https://image.aladin.co.kr/product/33010/94/co...,국내도서>외국어>토익>Reading,(주)YBM(와이비엠),10,10


In [15]:
test_book_lists = df_test['isbn13'].tolist()
test_book_lists

['9791138432283',
 '9788936434120',
 '9788954682152',
 '9791138928113',
 '9788936434595',
 '9791138928120',
 '9791142305320',
 '9791130646381',
 '9791138485593',
 '9788917239508']

In [33]:
test_book_id = df_aladin['itemId'].tolist()
test_book_id

[355241937,
 40869703,
 278770576,
 353981757,
 291370219,
 353981994,
 355678823,
 329386807,
 355242244,
 330109498]

## 줄거리 Crawling

In [79]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def initialize_driver(headless=True):
    """
    ChromeDriver를 초기화하는 함수.
    :param headless: True이면 브라우저 창을 표시하지 않음.
    :return: 초기화된 Chrome WebDriver 인스턴스.
    """
    # Chrome 옵션 설정
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # 브라우저 창 숨김
    chrome_options.add_argument("--disable-gpu")  # GPU 비활성화 (Linux에서 필수)
    chrome_options.add_argument("--no-sandbox")  # 샌드박스 비활성화
    chrome_options.add_argument("--disable-dev-shm-usage")  # 공유 메모리 크기 문제 해결
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    )  # 사용자 에이전트 설정

    # ChromeDriver 초기화
    service = Service(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

# WebDriver 시작
driver = initialize_driver(headless=False)  # headless=False로 변경하여 브라우저를 띄운 상태에서 확인

itemId = '40869703'
url = f"https://www.aladin.co.kr/shop/wproduct.aspx?ItemId={itemId}"

# URL로 이동
driver.get(url)

# 페이지가 완전히 로드될 때까지 대기
try:
    # 특정 요소가 로드될 때까지 대기 (예: 상품 제목이 있는 클래스 이름을 지정)
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "Ere_prod_title"))
    )

    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(page_source, "html.parser")

    # 상품 제목 가져오기
    title_element = soup.find("div", class_="Ere_prod_title")
    if title_element:
        product_title = title_element.get_text(strip=True)
        print(f"상품 제목: {product_title}")
    else:
        print("상품 제목을 찾을 수 없습니다.")

except Exception as e:
    print(f"에러 발생: {e}")

finally:
    # WebDriver 종료
    driver.quit()

에러 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0061FD53+23747]
	(No symbol) [0x005A7D54]
	(No symbol) [0x0047BE53]
	(No symbol) [0x004BFCA6]
	(No symbol) [0x004BFEEB]
	(No symbol) [0x004FD852]
	(No symbol) [0x004E1E44]
	(No symbol) [0x004FB41E]
	(No symbol) [0x004E1B96]
	(No symbol) [0x004B3F3C]
	(No symbol) [0x004B4EBD]
	GetHandleVerifier [0x008FAC73+3017699]
	GetHandleVerifier [0x0090B93B+3086507]
	GetHandleVerifier [0x009040F2+3055714]
	GetHandleVerifier [0x006B5AF0+637536]
	(No symbol) [0x005B0A5D]
	(No symbol) [0x005ADA28]
	(No symbol) [0x005ADBC5]
	(No symbol) [0x005A07F0]
	BaseThreadInitThunk [0x76C85D49+25]
	RtlInitializeExceptionChain [0x7739CEBB+107]
	RtlGetAppContainerNamedObjectPath [0x7739CE41+561]



# 카카오

In [25]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

# .env 파일 로드 (환경 변수에 API 키 설정)
load_dotenv()

# 환경 변수에서 KAKAO API 키 가져오기
KAKAO_API_KEY = os.environ.get("KAKAO_API_KEY")

# API 요청 URL
url = "https://dapi.kakao.com/v3/search/book"

# ISBN 리스트
isbn_list = test_book_lists

# 결과 저장 리스트
contents_list = []

# 요청 헤더 설정
headers = {
    "Authorization": f"KakaoAK {KAKAO_API_KEY}"
}

# 여러 ISBN에 대해 요청 반복
for isbn in isbn_list:
    params = {
        "target": "isbn",
        "query": isbn
    }
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        # 성공적으로 데이터를 받아온 경우
        data = response.json()
        if 'documents' in data and len(data['documents']) > 0:
            # 'contents'만 추출
            contents_list.extend([doc['contents'] for doc in data['documents']])
    else:
        # 요청 실패 로그
        print(f"ISBN {isbn} 요청 실패: {response.status_code}")
        print(response.text)

# DataFrame 생성
df_kakao = pd.DataFrame(contents_list, columns=['KAKAO_contents'])

df_kakao

,KAKAO_contents
0,"멤버를 모아, 팬텀 버스터즈로서 제령 활동을 시작한 모가리와 친구들. 인지도를 높이..."
1,"2014년 만해문학상, 2017년 이탈리아 말라파르테 문학상을 수상하고 전세계 20..."
2,2016년 『채식주의자』로 인터내셔널 부커상을 수상하고 2018년 『흰』으로 같은 ...
3,
4,2016년 인터내셔널 부커상을 수상하며 한국문학의 입지를 한단계 확장시킨 한강의 장...
5,
6,
7,"* 문학평론가 신형철, 르포작가 은유 추천 * 2022 오웰상 소설 부문 수상 * ..."
8,경성에서 연속 엽기 살인사건을 해결한 검은 옷의 무녀 최월자와 일본인 작가 이와야 ...
9,1. 실제 기출문제 전격 공개! 이 책에는 정기시험 최신 기출문제 10세트가 수록되...


# 네이버

In [24]:
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from dotenv import load_dotenv

# .env 파일 로드 (환경 변수에 API 키 설정)
load_dotenv()

# 환경 변수에서 Naver API 클라이언트 정보 가져오기
NAVER_CLIENT_ID = os.environ.get("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.environ.get("NAVER_CLIENT_SECRET")

# API 요청 URL
url = "https://openapi.naver.com/v1/search/book_adv.xml"

# ISBN 리스트
isbn_list = test_book_lists

# 결과 저장 리스트
descriptions = []

# 요청 헤더 설정
headers = {
    "X-Naver-Client-Id": NAVER_CLIENT_ID,
    "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
}

# 여러 ISBN에 대해 요청 반복
for isbn in isbn_list:
    params = {
        "d_isbn": isbn,  # ISBN 검색어
        "display": 10,   # 한 번에 가져올 결과 수
        "start": 1       # 검색 시작 위치
    }
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        # XML 파싱
        root = ET.fromstring(response.text)
        for item in root.findall('.//item'):
            description = item.find('description').text if item.find('description') is not None else ""
            descriptions.append(description)
    else:
        # 요청 실패 로그
        print(f"ISBN {isbn} 요청 실패: {response.status_code}")
        print(response.text)

# DataFrame 생성 (description만 포함)
df_naver = pd.DataFrame(descriptions, columns=["Naver_description"])

# 출력
df_naver

,Naver_description
0,SNS에서도 화제만발!\n본격적으로 활동을 시작한 팬버스의\n앞을 가로막는 자객과 ...
1,"말라파르테 문학상, 만해문학상 수상작 \n우리 시대의 소설 『소년이 온다』\n\n2..."
2,무엇을 생각하면 견딜 수 있나.\n가슴에 활활 일어나는 불이 없다면.\n기어이 돌아...
3,오랜 연구와 검증으로 한국사에 기초가 전혀 없는 사람도 단기간에 한능검에 합격할 수...
4,"인터내셔널 부커상, 산클레멘테 문학상 수상작\n전세계가 주목한 한강의 역작을 다시 ..."
5,&lt;2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화 하&gt;\n\n...
6,해적왕이라고 불리웠던 'G 로저'가 남긴 보물 중의 보물 &lt;원피스&gt;를 둘...
7,"한 세대에 한 명씩만 나오는 작가, 클레어 키건의 대표작!\n\n* 문학평론가 신형..."
8,크툴루풍x샤머니즘 퇴마 괴담!\n화제의 코리안 오컬틱 액션!!\n이번 무대는 제주도...
9,출제기관 ETS 토익 정기시험 기출문제 10회 최신판 Vol.4 독점공개!\n시험에...


# 구글

# 국립중앙도서관

## 통합 데이터

In [36]:
df_test = pd.concat([df_aladin[['title', 'description', 'itemId']], df_kakao, df_naver], axis=1)
df_test

,title,description,itemId,KAKAO_contents,Naver_description
0,팬텀 버스터즈 2 - S코믹스,"멤버를 모아, 팬텀 버스터즈로서 제령 활동을 시작한 모가리와 친구들. 인지도를 높...",355241937,"멤버를 모아, 팬텀 버스터즈로서 제령 활동을 시작한 모가리와 친구들. 인지도를 높이...",SNS에서도 화제만발!\n본격적으로 활동을 시작한 팬버스의\n앞을 가로막는 자객과 ...
1,소년이 온다 - 2024 노벨문학상 수상작가,섬세한 감수성과 치밀한 문장으로 인간 존재의 본질을 탐구해온 작가 한강의 여섯번째 ...,40869703,"2014년 만해문학상, 2017년 이탈리아 말라파르테 문학상을 수상하고 전세계 20...","말라파르테 문학상, 만해문학상 수상작 \n우리 시대의 소설 『소년이 온다』\n\n2..."
2,"작별하지 않는다 - 2024 노벨문학상 수상작가, 한강 장편소설",2016년 &lt;채식주의자&gt;로 인터내셔널 부커상을 수상하고 2018년 &lt...,278770576,2016년 『채식주의자』로 인터내셔널 부커상을 수상하고 2018년 『흰』으로 같은 ...,무엇을 생각하면 견딜 수 있나.\n가슴에 활활 일어나는 불이 없다면.\n기어이 돌아...
3,"2025 큰별쌤 최태성의 별★별한국사 한국사능력검정시험 심화(1, 2, 3급) 상 ...",오랜 연구와 검증으로 한국사에 기초가 전혀 없는 사람도 단기간에 한능검에 합격할 수...,353981757,,오랜 연구와 검증으로 한국사에 기초가 전혀 없는 사람도 단기간에 한능검에 합격할 수...
4,채식주의자 (리마스터판) - 2024 노벨문학상 수상작가,2016년 인터내셔널 부커상을 수상하며 한국문학의 입지를 한단계 확장시킨 한강의 장...,291370219,2016년 인터내셔널 부커상을 수상하며 한국문학의 입지를 한단계 확장시킨 한강의 장...,"인터내셔널 부커상, 산클레멘테 문학상 수상작\n전세계가 주목한 한강의 역작을 다시 ..."
5,"2025 큰별쌤 최태성의 별★별한국사 한국사능력검정시험 심화(1, 2, 3급) 하 ...",오랜 연구와 검증으로 한국사에 기초가 전혀 없는 사람도 단기간에 한능검에 합격할 수...,353981994,,&lt;2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화 하&gt;\n\n...
6,원피스 110 - 시대의 일렁임,"거병 해적단의 도움으로 미래섬 탈출에 빛줄기를 본 루피 일행이었으나, 오로성이 벽이...",355678823,,해적왕이라고 불리웠던 'G 로저'가 남긴 보물 중의 보물 &lt;원피스&gt;를 둘...
7,이처럼 사소한 것들,2023년 4월 국내에 처음 소개된 『맡겨진 소녀』로 국내 문인들과 문학 독자들의 ...,329386807,"* 문학평론가 신형철, 르포작가 은유 추천 * 2022 오웰상 소설 부문 수상 * ...","한 세대에 한 명씩만 나오는 작가, 클레어 키건의 대표작!\n\n* 문학평론가 신형..."
8,흑무경담 2 - S코믹스,경성에서 연속 엽기 살인사건을 해결한 검은 옷의 무녀 최월자와 일본인 작가 이와야 ...,355242244,경성에서 연속 엽기 살인사건을 해결한 검은 옷의 무녀 최월자와 일본인 작가 이와야 ...,크툴루풍x샤머니즘 퇴마 괴담!\n화제의 코리안 오컬틱 액션!!\n이번 무대는 제주도...
9,ETS 토익 정기시험 기출문제집 1000 Vol. 4 Reading (리딩) - 토...,정기시험 최신 기출문제 10세트가 수록되어 있다. 시험에 나온 최신 기출문제로 실전...,330109498,1. 실제 기출문제 전격 공개! 이 책에는 정기시험 최신 기출문제 10세트가 수록되...,출제기관 ETS 토익 정기시험 기출문제 10회 최신판 Vol.4 독점공개!\n시험에...


In [37]:
df_test.to_csv('test_data.csv', encoding='utf-8-sig', index=False)